In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import os
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.optim
from geographiclib.geodesic import Geodesic
# idw 需要的库
import geopandas as gpd
import math
from math import radians, sin, cos, asin, sqrt
from shapely.geometry import Polygon, Point
import sklearn.metrics as metrics
import random


# 我的需求
首先，它需要是一个回归模型，因为我想要预测每个点的超参数值。不仅是已知点，还包括未知点，那么我应该先随机得到一组未知点（使用buffer），然后对每一个点都计算一个超参数。  


我们先进行一次插值，然后再进行参数的计算，这样的话，我们就可以得到每一个未知点的value信息，这样的任务就变成了提高插值精度。

然后，输入数据是经度、纬度和降水量，这是一个三维的输入，一维的输出。但是，模型的结构还不知道。在这种情况下，可能需要考虑如何将带有空间信息的输入数据转换为适合的格式？



## 先计算一次插值，然后再进行参数的计算
这里使用的就是普通的IDW

In [ ]:

# 数据处理形成正态分布或0-1的样子，我们的降雨数据没有负数
def standardize(data):
    mean = np.mean(data)
    std = np.std(data)
    transformed_data = np.empty(data.shape)
    for i in range(len(data)):
        transformed_data[i] = (data[i] - mean) / std

    return transformed_data

def normalize(data):
    max = np.max(data)
    min = np.min(data)
    transformed_data = np.empty(data.shape)
    for i in range(len(data)):
        transformed_data[i] = (data[i] - min) / (max - min)

    return transformed_data

def caldis(lon1, lat1, lon2, lat2):  # 输入两点经纬度
    a = radians(lat1 - lat2)
    b = radians(lon1 - lon2)
    lat1, lat2 = radians(lat1), radians(lat2)
    t = sin(a / 2) ** 2 + cos(lat1) * cos(lat2) * sin(b / 2) ** 2
    distance = 2 * asin(sqrt(t)) * 6378.137

    # 设置一个阈值，当距离接近于零时，使用一个非零的默认距离值,防止溢出
    threshold = 1e-6
    if distance < threshold:
        distance = threshold

    return distance  # 返回两点距离

# parameter是一个超参数，用于调整插值的精度，默认为1，这样不影响没插值时的计算
# 我们在这里预设获得的参数排列顺序和输入的点排列顺序是一致的，那么实际上我要在之前进行一个对应的排序
def idw(lon, lat, value, x, y, parameter = 1):
    prediction_result = []
    for i in range(len(x)):
        distance_list = []
        for j in range(len(lon)):
            distance = caldis(lon[j], lat[j], x[i], y[i])
            distance_list.append(distance)
        # 我需要找到这两个点的超参数，然后再进行插值
        sqdis = list(1 / np.power(distance_list, 2))

        # 每个点的已知值 * 每个点的权重（未知点到每个已知点的距离） * 每个点的超参数，由于都是相同大小，所以直接相乘
        z = np.sum(np.array(value) * np.array(sqdis) * np.array(parameter)) / np.sum(sqdis)
        prediction_result.append(z)

    return prediction_result  # 返回每个点插值组成的列表

# 在缓冲区内随机生成点
def random_point_within_polygon(polygon):
    min_x, min_y, max_x, max_y = polygon
    # 创建了一个具有四个角点的多边形
    polygon_obj = Polygon([(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)])  # Create a Polygon object
    while True:
        random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        if polygon_obj.contains(random_point):
            return random_point



生成随机点，然后进行插值，并计算误差

In [ ]:
# 先读取数据（实际上的文件里面有近几十年来每一年的降水量，不过我们并没有用到）
data = pd.read_csv("/content/drive/MyDrive/geo-master/random_data.csv", usecols=['lat', 'lon', '2016'])
lat, lon, value = data.lat, data.lon, data['2016']
value = normalize(value)


# 准备存储随机点的经纬度，已知点信息，随机点信息
points = gpd.GeoDataFrame()
random_points = gpd.GeoDataFrame()
prediction_result = []

# 读取这个点（point），生成一个buffer，然后在其中生成一个随机点，这些随机点被存在random_points中
for i in range(len(lat)):
    # 存储点到geodataframe中
    points = pd.concat([points, gpd.GeoDataFrame(geometry=[Point(lon[i], lat[i])])], ignore_index=True)

    # 创建buffer
    circle_buffer = Point(lon[i], lat[i]).buffer(0.5)

    # 生成随机点的个数，1表示每一个已知点周围只生成一个随机点
    # for j in range(1):
    random_point = random_point_within_polygon(circle_buffer.bounds)

        # 存储随机点，随机点的经纬度也存储在geodataframe中了
    random_points = pd.concat([random_points, gpd.GeoDataFrame(geometry=[random_point])], ignore_index=True)

# 输入数据集，每个样本包含经度、纬度和降水量，读入pd的dataframe中
# 用idw插值
prediction_result = idw(lon.tolist(), lat.tolist(), value, random_points.geometry.x, random_points.geometry.y)
#print(prediction_result)


# 计算准确性
mae = metrics.mean_absolute_error(value, prediction_result)  # 0 表示完美预测，值越大表示预测误差越大。
mse = metrics.mean_squared_error(value, prediction_result)  # 0 表示完美预测，值越大表示预测误差越大。
rmse = np.sqrt(mse)
r2 = metrics.r2_score(value, prediction_result)  # 1 表示完美预测，0 表示模型与简单平均值的效果相同，负值表示模型预测比直接使用平均值还要差。

print("The accuracy of mae, mse, rmse, r2:", (mae, mse, rmse, r2))


## 实现一个处理空间数据的Transformer
1. 读取数据集，然后将**数据集转换为tensor**，然后将数据集分为训练集和验证集，最后使用DataLoader加载数据集。

In [ ]:
# 数据集
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self): # 其实我感觉在这段代码中，不用这些好像也可以
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample


# 数据集需要转换为tensor
data = np.array(data)
data = torch.tensor(data).float()

# 添加一个维度作为seq_lenth，这里还没有特别完整的方案
data = data.unsqueeze(1)

dataset = MyDataset(data)
# 这时候的data是[1000， 3]，1000个样本，3个特征

# 分数据集为训练，验证
train_size = int(len(data) * 0.5)
validate_size = int(len(data) * 0.2)
test_size = len(data) - validate_size - train_size
train_dataset, validate_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, validate_size, test_size])

# 使用DataLoader加载数据
batch_size = 8 # 这个怎么选？
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)# 那么我输入时候应该是许多个[16, 3]
validate_loader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

'''
data_iter = iter(train_loader)
batch = next(data_iter)
print("Data shape in the batch:", batch.shape)
'''

在[文章](https://arxiv.org/abs/2311.15530)中，能使用transformer处理空间数据，它的做法有
1. 将原始数据输入模型时，经过两层全连接，作为input embedding；
2. 位置嵌入时，我们根据经纬度获取两两距离和角度矩阵，而不是使用sin cos 的绝对位置；
3. 使用上面的位置矩阵，和qk相乘，作为注意力；
于是我仿照它处理空间信息的方式，创建New_Transformer模块。

新：
使用KMeans将点数据划分为patch。使用零填充不足的部分，或者用平均值。
计算每个patch的中心位置和每个点在patch中的相对位置。用于embedding，并删除initial_embedding。删除pos_matrix(bs, bs, 2).（实际上是把它换成了patches）
转换为Torch后输入。
那我有一个问题，这个batch_size还怎么确定？
在原来的方法中，我记录了batch中的点的距离和角度。同样的，我在这里也需要记录每个patch中的点的信息。

In [ ]:
from sklearn.cluster import KMeans

# 将data的点划分为多个patch：仅使用经纬度信息进行kmeans，或者使用GNN?
n_patches = 10
kmeans = KMeans(n_clusters=n_patches)
patch_labels = kmeans.fit_predict(data[:, :2])  # ndarray of shape (n_samples,) 包含每一堆的标签

# 创建patch并计算位置嵌入，选择所有属于第i个patch的点。选择这个patch的中心点，然后计算每个点相对于中心点的相对位置。
patches = []
patch_centers = []
for i in range(n_patches):
    patch = data[patch_labels == i]
    patches.append(patch)
    patch_center = patch[:, :2].mean(axis=0)
    patch_centers.append(patch_center)

# 确保每个patch的大小相同（补零或其他处理）
max_patch_size = max(len(patch) for patch in patches)
padded_patches = []
relative_positions = []

for patch, center in zip(patches, patch_centers):
    padding = np.zeros((max_patch_size - len(patch), data.shape[1]))
    padded_patch = np.vstack((patch, padding))
    padded_patches.append(padded_patch)
    
    relative_pos = np.zeros((max_patch_size, 2))
    relative_pos[:len(patch), :] = patch[:, :2] - center
    relative_positions.append(relative_pos)
    
    
# 转换为tensor
patches = torch.tensor(padded_patches, dtype=torch.float32)
relative_pos = torch.tensor(relative_pos, dtype=torch.float32)
patch_centers = torch.tensor(patch_centers, dtype=torch.float32)



修改了一下calc_dist_angle_mat，但是之后不一定会用到

In [ ]:

# 输出一个距离和角度矩阵
def calc_dist_angle_mat(data):

    # 输入经纬度
    # dist_angle_mat [bs, bs, 2]
    # 它遍历每一对点，因此对于1000个点也是比较大的数据,1000*1000次运算
    # 这里的data是一个batch中的部分，所以不需要我手动筛选
    lat = data.lat
    lon = data.lon
    lats = data.index_select(-1, torch.tensor([0]))
    lons = data.index_select(-1, torch.tensor([1]))

    dist_angle_mat = np.zeros((len(lons), len(lons), 2))

    for i in range(len(lons)):
        for j in range(len(lons)):
            dist = Geodesic.WGS84.Inverse(lats[i], lons[i], lats[j], lons[j])
            dist_angle_mat[i, j, 0] = dist["s12"] / 1000.0  # distance, km
            dist_angle_mat[i, j, 1] = dist["azi1"]  # azimuth at the first point in degrees

    #print(dist_angle_mat.shape)
    # print(dist_angle_mat)

    return dist_angle_mat
    
def pos_mat(data):

    # 同样是bs, bs, 2 但是用两个值表示位置
    # 但是这里的data是一个batch的话，我需要筛选才能得到每个点的信息，现在两个办法：
    # 1.筛选 2.不用bs了，改用patch作为一组
    pos_mat = np.zeros((len(data), len(data), 2))
    patch_count = 
    patch
    
    return pos_mat
# 定义一个active函数
def gelu(x):
   return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))
# 2个全连接
class TwoLayerFCN(nn.Module):
    def __init__(self, feat_dim, n_hidden1, n_hidden2):
        super().__init__()
        self.feat_dim = feat_dim
        self.linear_1 = nn.Linear(feat_dim, n_hidden1)
        self.linear_2 = nn.Linear(n_hidden1, n_hidden2)

    def forward(self, in_vec, non_linear=False):
        """pos_vec: absolute position vector, n * feat_dim"""
        assert in_vec.shape[-1] == self.feat_dim, f"in_vec.shape: {in_vec.shape}, feat_dim:{self.feat_dim}"

        if non_linear:
            mid_emb = F.relu(self.linear_1(in_vec))
        else:
            mid_emb = self.linear_1(in_vec)

        output = self.linear_2(mid_emb)
        return output
# qkc相乘，得到attention的输出
class DotProductAttention(nn.Module):# qkc相乘，得到attention的输出
    ''' attn: sum over element-wise product of three vectors'''
    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()

        #self.temperature = torch.tensor(temperature, dtype=torch.float)
        self.temperature =temperature
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, q, k, v, pos_mat, d_k, d_v, n_head, mask=None):
        # pos_mat = [bs, bs, 2] 这里输入的qkv = [bs, d_model][8, 16]
        batch_size, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)

        # Transpose for attention dot product: batch_size x n_head x len_q x dv
        # Separate different heads: batch_size x len_q x n_head x dv

        r_q1 = q.view(batch_size, len_q, n_head, d_k).permute(0, 2, 1, 3) # torch.Size([8, 2, 1, 16])
        r_k1 = k.view(batch_size, len_q, n_head, d_k).permute(0, 2, 1, 3)
        r_v1 = v.view(batch_size, len_v, n_head, d_v).permute(0, 2, 1, 3) # torch.Size([8, 2, 1, 16])

        attn1 = torch.mul(r_q1.unsqueeze(2), r_k1.unsqueeze(3))# attn1: [batch_size, n_head, len_q, len_q, d_k], c: [bs, bs, d_k]

        attn = torch.mul(attn1, pos_mat)  # attn: [batch_size, n_head, len_q, len_q, d_k]  torch.Size([8, 2, 8, 8, 16])
        attn = torch.sum(attn, -2)  # attn: [bs, n_head, len_q, len_q]  torch.Size([8, 2, 8, 8])

        torch.div(attn, self.temperature) # 除以根号dk

        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e10)

        attn = self.dropout(F.softmax(attn, dim=-1))
        # attn: [bs, n_head, len_q, len_q] r_v1: [bs, n_head, len_v, d_v]
        # torch.Size([8, 2, 8, 8]) torch.Size([8, 2, 1, 16])
        # print(attn.shape, r_v1.shape)
        output = torch.mul(attn, r_v1)

        return output, attn
# 多头注意力
class RelativeMultiHeadAttention(nn.Module):
    def __init__(self, n_head, d_model, d_k, d_v, temperature=None, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_v

        if temperature is None:
            temperature = d_k ** 0.5

        # 将RelativePosition类的初始化部分复制到这里
        self.linear_1 = nn.Linear(2 , d_k)
        self.linear_2 = nn.Linear(d_k, d_k)

        self.attention = DotProductAttention(temperature=temperature)
        # w_qs, w_ks, w_vs: [d_model, n_head * d_k/d_v]
        # w_qs表示对q进行线性变换，
        self.w_qs = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_ks = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_vs = nn.Linear(d_model, n_head * d_v, bias=False)
        self.fc = nn.Linear(n_head * d_v, d_model, bias=False)

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)

    def forward(self, q, k, v, pos_mat, mask=None):
        d_k, d_v, d_model, n_head = self.d_k, self.d_v, self.d_model, self.n_head
        sz_b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)
        
        residual = q

        q = self.w_qs(q)
        k = self.w_ks(k)
        v = self.w_vs(v)


        # 相对位置嵌入
        n_element = pos_mat.shape[0]
        positions = torch.tensor(pos_mat.reshape(-1), dtype=torch.float)
        a_k = self.linear_2(self.linear_1(positions)).view(n_element, n_element, -1) # from [bs, bs, 2] to [bs, bs, d_k]

        if mask is not None:  # used to achieve Shielded Attention
            mask = mask.unsqueeze(1)  # For head axis broadcasting.

        q, attn = self.attention(q, k, v, a_k, d_k, d_v, n_head, mask=mask)

        # Transpose to move the head dimension back: sz_b x len_q x n_head x dv
        # Combine the last two dimensions to concatenate all the heads together: sz_b x len_q x (n_head*dv)
        q = q.transpose(1, 2).contiguous().view(sz_b, -1, n_head * d_v)  # torch.Size([8, 8, 2*16])
        q = self.dropout(self.fc(q))
        q += residual

        q = self.layer_norm(q)

        return q, attn
#位置前馈
class PositionWiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''
    def __init__(self, d_in, d_hid, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid)  # position-wise
        self.w_2 = nn.Linear(d_hid, d_in)  # position-wise
        self.layer_norm = nn.LayerNorm(d_in, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x

        x = self.w_2(F.relu(self.w_1(x)))
        x = self.dropout(x)
        x += residual

        x = self.layer_norm(x)

        return x
# 模型主要的部分
class New_TransFormer(nn.Module):
    def __init__(self, d_k, d_v, num_head, num_layer, d_model, d_feature, dropout=0.1, scale_emb=False, temperature=None, num_patches=10):
        super().__init__()

        self.d_model = d_model
        self.scale_emb = scale_emb
        self.d_k = d_k

        #self.initial_embedding = TwoLayerFCN(d_feature, d_model, d_model)
                # Positional embedding
        self.patch_pos_embed = nn.Parameter(torch.ones(1, num_patches, d_model)) # 因为patch没有一个基准，所以创建一个参数矩阵，来表示patch的位置？
        self.relative_pos_embed = nn.Linear(2, d_model)
        
        self.dropout = nn.Dropout(p=dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)

        # encoder
        self.MultiheadAttention = nn.ModuleList([RelativeMultiHeadAttention(num_head, d_model, d_k, d_v, temperature, dropout=dropout) for _ in range(num_layer)])
        self.feedforward = nn.ModuleList([PositionWiseFeedForward(d_model, d_model, dropout=dropout) for _ in range(num_layer)])

        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu

        self.decoder = TwoLayerFCN(d_model, d_model, 1)



    def forward(self, data, patches, relative_positions):
        B, N, T, _ = patches.shape
        # Flatten patches and apply linear projection
        patches = patches.view(B * N, T, -1)
        patches = self.projection(patches)
        
        # 两个部分，一个是patch在整体中的位置，一个是点在patch中的位置
        patch_pos_embedding = self.patch_pos_embed.expand(B, -1, -1)
        relative_pos_embedding = self.relative_pos_embed(relative_positions.view(B * N, T, -1))
        output = patches + patch_pos_embedding.view(B * N, 1, -1) + relative_pos_embedding
        
    
    
        #output = self.initial_embedding(data) # from [bs, d_features] to [bs, d_model]
        # 接下来经过n_layers层的encoder层，一个encoder 包括一个attention和一个feedforward
        # 先计算出距离和角度矩阵，等一下要进行位置嵌入和对q,k相乘，作为注意力
        pos_mat = normalize(calc_dist_angle_mat(data))  #  [bs, bs, 2]

        if self.scale_emb:  # 这个原理不是很懂
            output *= self.d_model ** 0.5

        output = self.dropout(output)
        output = self.layer_norm(output)


        #output = output + pos_mat  # 形状不一样

        # 重复i次注意力
        for i in range(len(self.MultiheadAttention)):
            output, _ = self.MultiheadAttention[i](output, output, output, pos_mat, mask=None)
            output = self.feedforward[i](output)


        # encoder结束
        """
        原始的mask部分
                masked_pos = masked_pos[:, :, None].expand(-1, -1, enc_output.size(-1))  # [batch_size, max_pred, d_model]

        # get masked position from final output of transformer.
        h_masked_1 = torch.gather(enc_output, 1, masked_pos)  # masking position [batch_size, max_pred, d_model]
        h_masked_2 = self.layer_norm(self.activ2(self.linear(h_masked_1)))
        dec_output = self.decoder(h_masked_2)  # [batch_size, max_pred, n_vocab]

        """
        transformed_output = self.layer_norm(self.activ2(self.linear(output)))
        dec_output = self.decoder(transformed_output).squeeze(1)  # [batch_size, len_q, d_model] to [batch_size, d_model]

        return dec_output



## 开始训练



In [ ]:

# 模型实例化
model = New_TransFormer(d_k=16, d_v=16, num_head=2, num_layer=3, d_model=16, d_feature=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.08)
criterion = nn.MSELoss()
# print(model)


# 训练模型
num_epochs = 100

for epoch in range(num_epochs):
    for data in train_loader: # 对每个batch

        # 1. forward the model
        optimizer.zero_grad()
        outputs = model(data)

        # 2. MSE loss of predicting masked elements
        loss = criterion(outputs, data) # (result[3], data[3])

        # 3. backward and optimization only in train
        loss.backward()
        # optimizer.step()
        optimizer.step()

        # loss
        # running_loss += loss.item()
        #tot_loss += loss.item()

    #if epoch  % 5 == 0:
    epoch_loss = loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

## 使用更新后的参数进行插值
这里的IDW部分的改进包括：
    1. 对数据提前处理
    2. 不对每个网格点插值，而是对每个已知点周围的随机点进行插值。具体是生成一个buffer，然后在其中生成一个随机点，然后对这个随机点进行插值。
    3. 增加使用之前计算出的超参数值，更好地预测。

下面从已经训练好的模型中提取特征

In [ ]:
# 设置模型为评估模式
model.eval()

# 定义一个函数来提取编码器的特征并转换为列表

features_list = []
with torch.no_grad():
    for i in data:
        # 前向传播，提取特征
        features = model.encoder(i)
        # 将特征添加到列表中
        features_list.append(features)
# 将列表中的张量拼接为一个张量
features_tensor = torch.cat(features_list, dim=0)
# 将特征张量转换为列表
features_list = features_tensor.tolist()


# 提取特征并转换为列表
extracted_features = extract_features(data)


In [ ]:

# 再次用idw插值,不过这次加入了参数
prediction_result = idw(lon.tolist(), lat.tolist(), value, random_points.geometry.x, random_points.geometry.y, parameter = extracted_features)
print(prediction_result)



mae = metrics.mean_absolute_error(value, prediction_result)  # 0 表示完美预测，值越大表示预测误差越大。
mse = metrics.mean_squared_error(value, prediction_result)  # 0 表示完美预测，值越大表示预测误差越大。
rmse = np.sqrt(mse)
r2 = metrics.r2_score(value, prediction_result)  # 1 表示完美预测，0 表示模型与简单平均值的效果相同，负值表示模型预测比直接使用平均值还要差。

print("The accuracy of mae, mse, rmse, r2:", (mae, mse, rmse, r2))
